In [16]:
import joblib
import time
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
# read data
data_df = pd.read_csv(r"D:\code\data\iFEMG_data_set\One_channel_dataset\20221007.csv", index_col=0)
print(data_df.shape)
data_df.head()

(2155, 15)


,subject_name,height(cm),weight(kg),gender,age,sensor_channel,label(kg),FMG_increase,mav,rms,wave_length,zero_crossing,slope_sign_change,mean_freq,mean_power_freq
0,Wang Junhan,170,60,1,21,bicps_br,0.0,0.635723,10.453135,17.756081,3.921375,-0.437357,-0.245760,52.549982,57.153273
1,Wang Junhan,170,60,1,21,bicps_br,0.0,0.669451,2.126425,2.229017,0.939814,-0.458228,-0.143381,68.783704,91.455915
2,Wang Junhan,170,60,1,21,bicps_br,0.0,0.799892,1.900349,1.776664,1.317730,-0.397235,-0.123461,66.730040,91.044406
3,Wang Junhan,170,60,1,21,bicps_br,0.0,0.638235,1.742331,1.789428,0.926630,-0.372496,-0.129632,71.406150,96.157089
4,Wang Junhan,170,60,1,21,bicps_br,0.0,0.661840,2.419714,2.047832,1.420618,-0.386364,-0.172241,85.291018,106.993184


In [13]:
"""
data cleaning:
    1. NaN value
    2. label is MVC
"""
# replace 'MVC' with NaN
data_df = data_df.replace('MVC', np.nan)
# delete NaN value
data_df = data_df.dropna(how = 'any')

# label: str to num
data_df['label(kg)']= pd.to_numeric(data_df['label(kg)'])

print(data_df.shape)
data_df.dtypes


(1949, 15)


subject_name          object
height(cm)             int64
weight(kg)             int64
gender                 int64
age                    int64
sensor_channel        object
label(kg)            float64
FMG_increase         float64
mav                  float64
rms                  float64
wave_length          float64
zero_crossing        float64
slope_sign_change    float64
mean_freq            float64
mean_power_freq      float64
dtype: object

In [14]:
"""
1. get x and y data
2. split train and test
3. scale
"""
x_data = data_df[['height(cm)', 'weight(kg)', 'gender', 'age', 'label(kg)', 'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values
y_data = data_df['label(kg)'].values

train_data_r, test_data_r, train_label, test_label = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0, stratify = y_data)
# train_data = preprocessing.normalize(train_data_r, norm = 'max')
# test_data = preprocessing.normalize(test_data_r, norm = 'max')
scaler = preprocessing.StandardScaler().fit(train_data_r)
train_data = scaler.transform(train_data_r)
test_data = scaler.transform(test_data_r)

print("train data shape: ", train_data.shape)
print("train label length: ", train_label.shape)
print("test data shape: ", test_data.shape)
print("test label length: ", test_label.shape)

In [ ]:
# SVR回归模型

# 记录开始训练时间
start_time = time.time()
print("start training, time: ", time.strftime("%Y--%m--%d %H:%M:%S", time.localtime(start_time)))

# 自动选择合适的参数
svr = GridSearchCV(SVR(), param_grid={"kernel": ("rbf", "linear"), "C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
svr.fit(train_data, train_label)

print(svr.best_params_)

end_time = time.time()
print("finish, time: ", time.strftime("%Y--%m--%d %H:%M:%S", time.localtime(end_time)))

In [ ]:

score_test = svr.score(test_data, test_label)
score_train = svr.score(train_data, train_label)
print(str(svr) + "train score： " + str(score_train))
print(str(svr) + "test score： " + str(score_test))
test_pre = svr.predict(test_data)
train_pre = svr.predict(train_data)
print("train mean squared error: ", mean_squared_error(train_label, train_pre))
print("test mean squared error: ", mean_squared_error(test_label, test_pre))


In [ ]:
# 可独立运行的SVRmodel

# 使用boolean值索引出每种label对应的实验数据
# 排除第一列受试者名称和第二列label字符串
d0_data = all_feature_df.loc[all_feature_df.loc[:, 'label'] == 'd0'].values[:, 2:]
d2_data = all_feature_df.loc[all_feature_df.loc[:, 'label'] == 'd2'].values[:, 2:]
d5_data = all_feature_df.loc[all_feature_df.loc[:, 'label'] == 'd5'].values[:, 2:]
d7_data = all_feature_df.loc[all_feature_df.loc[:, 'label'] == 'd7'].values[:, 2:]
dm_data = all_feature_df.loc[all_feature_df.loc[:, 'label'] == 'dm'].values[:, 2:]

# 创建数据集和label
f_data = []
label = []
#    data = []
#    l_sit = np.zeros((f_sit.shape[0], 1))
#    l_stand = np.zeros((f_stand.shape[0], 1))
#    l_walk = np.zeros((f_walk.shape[0], 1))
#    l_stairA = np.zeros((f_stairA.shape[0], 1))
#    l_stairD = np.zeros((f_stairD.shape[0], 1))
#    l_rampA = np.zeros((f_rampA.shape[0], 1))
#    l_rampD = np.zeros((f_rampD.shape[0], 1))

for i in range(d0_data.shape[0]):
    label.append(0)

for i in range(d2_data.shape[0]):
    label.append(1)

for i in range(d5_data.shape[0]):
    label.append(2)

for i in range(d7_data.shape[0]):
    label.append(3)

for i in range(dm_data.shape[0]):
    label.append(4)

f_data = np.concatenate((d0_data, d2_data, d5_data, d7_data, dm_data), axis = 0)
print("data shape： ", f_data.shape)
print("label num: ", len(label))

# 划分训练集和测试集
train_data_r, test_data_r, train_label, test_label = train_test_split(f_data, label, test_size = 0.2, random_state = 0, stratify = label)
# train_data = preprocessing.normalize(train_data_r, norm = 'max')
# test_data = preprocessing.normalize(test_data_r, norm = 'max')
scaler = preprocessing.StandardScaler().fit(train_data_r)
train_data = scaler.transform(train_data_r)
test_data = scaler.transform(test_data_r)

# 记录开始训练时间
start_time = time.time()
print("start training, time: ", time.strftime("%Y--%m--%d %H:%M:%S", time.localtime(start_time)))


regression_model = SVR(kernel='rbf', C=1000, gamma=1)
regression_model.fit(train_data, train_label)

end_time = time.time()
print("finish, time: ", time.strftime("%Y--%m--%d %H:%M:%S", time.localtime(end_time)))

In [ ]:
joblib.dump(regression_model, 'OneChannelRegression.pkl')